In [117]:
%%time

import numpy as np
import pandas as pd

FILE_PATH = "/home/ubuntu/dos-californium/deter/expdata/real/final/client_and_attacker_httpson/1/proxy_dump.pcap.out"

p = FILE_PATH.split("/")[-1]
p = p[:p.index("_")]
node_type = "originserver" if p == "server" else p

dtype = {
}

df = pd.read_csv(FILE_PATH, engine="pyarrow", dtype=dtype, delimiter=";", quotechar='"')
df["node_type"] = node_type

df

CPU times: user 1.77 s, sys: 902 ms, total: 2.67 s
Wall time: 1.17 s


,_ws.col.Time,_ws.col.Source,_ws.col.Destination,_ws.col.Protocol,_ws.col.Length,coap.type,coap.retransmitted,coap.code,coap.mid,coap.token,coap.opt.proxy_uri,http.request,http.request.method,http.request.full_uri,http.response,http.response.code,http.response.code.desc,http.response_for.uri,node_type
0,1.647910e+09,10.1.5.1,10.1.5.99,CoAP,106,0.0,NaN,1.0,1.0,1d8bc74f5569de04,https://10.1.1.1:443/1_1D8BC74F5569DE04,NaN,,,NaN,NaN,,,proxy
1,1.647910e+09,10.1.5.99,10.1.1.1,HTTP,230,NaN,NaN,NaN,NaN,,,1.0,GET,https://10.1.1.1:443/1_1D8BC74F5569DE04,NaN,NaN,,,proxy
2,1.647910e+09,10.1.1.1,10.1.5.99,HTTP,491,NaN,NaN,NaN,NaN,,,NaN,,,1.0,200.0,OK,https://10.1.1.1:443/1_1D8BC74F5569DE04,proxy
3,1.647910e+09,10.1.5.99,10.1.5.1,CoAP,158,2.0,NaN,69.0,1.0,1d8bc74f5569de04,,NaN,,,NaN,NaN,,,proxy
4,1.647910e+09,10.1.5.1,10.1.5.99,CoAP,106,0.0,NaN,1.0,2.0,473bbaacedb0fbb2,https://10.1.1.1:443/2_473BBAACEDB0FBB2,NaN,,,NaN,NaN,,,proxy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964694,1.647911e+09,10.1.5.99,10.1.5.1,CoAP,158,2.0,NaN,69.0,1388.0,f774a0ae8029a854,,NaN,,,NaN,NaN,,,proxy
964695,1.647911e+09,10.1.5.1,10.1.5.99,CoAP,109,0.0,NaN,1.0,1389.0,033bbfae19e7c845,https://10.1.1.1:443/1389_033BBFAE19E7C845,NaN,,,NaN,NaN,,,proxy
964696,1.647911e+09,10.1.5.99,10.1.1.1,HTTP,233,NaN,NaN,NaN,NaN,,,1.0,GET,https://10.1.1.1:443/1389_033BBFAE19E7C845,NaN,NaN,,,proxy
964697,1.647911e+09,10.1.1.1,10.1.5.99,HTTP,491,NaN,NaN,NaN,NaN,,,NaN,,,1.0,200.0,OK,https://10.1.1.1:443/1389_033BBFAE19E7C845,proxy


In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 964699 entries, 0 to 964698
Data columns (total 19 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   _ws.col.Time             964699 non-null  float64
 1   _ws.col.Source           964699 non-null  object 
 2   _ws.col.Destination      964699 non-null  object 
 3   _ws.col.Protocol         964699 non-null  object 
 4   _ws.col.Length           964699 non-null  int64  
 5   coap.type                933115 non-null  float64
 6   coap.retransmitted       5 non-null       float64
 7   coap.code                933115 non-null  float64
 8   coap.mid                 933115 non-null  float64
 9   coap.token               964699 non-null  object 
 10  coap.opt.proxy_uri       964699 non-null  object 
 11  http.request             15792 non-null   float64
 12  http.request.method      964699 non-null  object 
 13  http.request.full_uri    964699 non-null  object 
 14  http

In [120]:
data_row_name_map_field_name = {
  "_ws.col.Time"       : "message_timestamp",
  "_ws.col.Source"     : "message_source",
  "_ws.col.Destination": "message_destination",
  "_ws.col.Protocol"   : "message_protocol",
  "_ws.col.Length"     : "message_size",
  "coap.type"          : "coap_type",
  "coap.code"          : "coap_code",
  "coap.mid"           : "coap_message_id",
  "coap.token"         : "coap_token",
  "coap.opt.proxy_uri" : "coap_proxy_uri",
  "coap.retransmitted" : "coap_retransmitted",
}

ip_addr_map_host_name = {
    '10.1.1.1': 'originserver',
    '10.1.2.1': 'receiver',
    '10.1.3.1': 'attacker',
    '10.1.5.1': 'client1',
    '10.1.5.99': 'proxy',
}

type_map_text = {
  0: "con",
  1: "non",
  2: "ack",
  3: "rst",
}

coap_code_to_string = lambda c, dd : (c << 5) | dd
code_map_text = {
  # Empty Message
  coap_code_to_string(0, 00) : "empty_message",
  
  # Method Codes
  coap_code_to_string(0,  1) : "get",
  coap_code_to_string(0,  2) : "put",
  coap_code_to_string(0,  3) : "post",
  coap_code_to_string(0,  4) : "delete",

  # Response Codes
  coap_code_to_string(2,  1) : "created",
  coap_code_to_string(2,  2) : "deleted",
  coap_code_to_string(2,  3) : "valid",
  coap_code_to_string(2,  4) : "changed",
  coap_code_to_string(2,  5) : "content",
  coap_code_to_string(4, 00) : "bad_request",
  coap_code_to_string(4,  1) : "unauthorized",
  coap_code_to_string(4,  2) : "bad_option",
  coap_code_to_string(4,  3) : "forbidden",
  coap_code_to_string(4,  4) : "not_found",
  coap_code_to_string(4,  5) : "method_not_allowed",
  coap_code_to_string(4,  6) : "not_acceptable",
  coap_code_to_string(4, 12) : "precondition_failed",
  coap_code_to_string(4, 13) : "request_entity_too_large",
  coap_code_to_string(4, 15) : "unsupported_media_type",
  coap_code_to_string(5, 00) : "internal_server_error",
  coap_code_to_string(5,  1) : "not_implemented",
  coap_code_to_string(5,  2) : "bad_gateway",
  coap_code_to_string(5,  3) : "service_unavailable",
  coap_code_to_string(5,  4) : "gateway_timeout",
  coap_code_to_string(5,  5) : "proxying_not_supported",

  # Everything else is reserved
}

In [121]:
%%time

# Rename column names from Wireshark format to database format
df.rename(columns=data_row_name_map_field_name, inplace=True)

# Replace IP addresses with host names
df.replace({"message_source": ip_addr_map_host_name, "message_destination": ip_addr_map_host_name}, inplace=True)

# Lowercase protocol names
df["message_protocol"] = df["message_protocol"].str.lower()

df

CPU times: user 1.02 s, sys: 296 ms, total: 1.32 s
Wall time: 1.32 s


,message_timestamp,message_source,message_destination,message_protocol,message_size,coap_type,coap_retransmitted,coap_code,coap_message_id,coap_token,coap_proxy_uri,http.request,http.request.method,http.request.full_uri,http.response,http.response.code,http.response.code.desc,http.response_for.uri,node_type
0,1.647910e+09,client1,proxy,coap,106,0.0,NaN,1.0,1.0,1d8bc74f5569de04,https://10.1.1.1:443/1_1D8BC74F5569DE04,NaN,,,NaN,NaN,,,proxy
1,1.647910e+09,proxy,originserver,http,230,NaN,NaN,NaN,NaN,,,1.0,GET,https://10.1.1.1:443/1_1D8BC74F5569DE04,NaN,NaN,,,proxy
2,1.647910e+09,originserver,proxy,http,491,NaN,NaN,NaN,NaN,,,NaN,,,1.0,200.0,OK,https://10.1.1.1:443/1_1D8BC74F5569DE04,proxy
3,1.647910e+09,proxy,client1,coap,158,2.0,NaN,69.0,1.0,1d8bc74f5569de04,,NaN,,,NaN,NaN,,,proxy
4,1.647910e+09,client1,proxy,coap,106,0.0,NaN,1.0,2.0,473bbaacedb0fbb2,https://10.1.1.1:443/2_473BBAACEDB0FBB2,NaN,,,NaN,NaN,,,proxy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964694,1.647911e+09,proxy,client1,coap,158,2.0,NaN,69.0,1388.0,f774a0ae8029a854,,NaN,,,NaN,NaN,,,proxy
964695,1.647911e+09,client1,proxy,coap,109,0.0,NaN,1.0,1389.0,033bbfae19e7c845,https://10.1.1.1:443/1389_033BBFAE19E7C845,NaN,,,NaN,NaN,,,proxy
964696,1.647911e+09,proxy,originserver,http,233,NaN,NaN,NaN,NaN,,,1.0,GET,https://10.1.1.1:443/1389_033BBFAE19E7C845,NaN,NaN,,,proxy
964697,1.647911e+09,originserver,proxy,http,491,NaN,NaN,NaN,NaN,,,NaN,,,1.0,200.0,OK,https://10.1.1.1:443/1389_033BBFAE19E7C845,proxy


In [122]:
%%time

# TODO: When done testing, switch cdf back to df and remove the copy assignment

coap_messages = (df["message_protocol"] == "coap")
cdf = df.copy()[coap_messages]

# Convert coap type integer to human readable string
cdf.loc[coap_messages, "coap_type"] = cdf.loc[coap_messages]["coap_type"].astype(np.uint16).replace(type_map_text)

# Convert coap code integer to human readable string
cdf.loc[coap_messages, "coap_code"] = cdf.loc[coap_messages]["coap_code"].astype(np.uint16).replace(code_map_text)

# Sometimes there is a random : so we need to remove it
cdf["coap_token"] = cdf["coap_token"].str.replace(":", "")

# Convert coap retransmitted to a boolean
cdf.loc[cdf["coap_retransmitted"].notna(), "coap_retransmitted"] = True
cdf.loc[cdf["coap_retransmitted"].isna(), "coap_retransmitted"] = False
cdf["coap_retransmitted"] = cdf["coap_retransmitted"].astype(np.bool_)

# Assign each message a UID
cdf["uid"] = cdf["coap_message_id"].astype(str) + "_" + cdf["coap_token"].astype(str)

# Validators
assert len({True, False} - set(cdf["coap_retransmitted"].unique())) == 0 # Nothing but True, False

cdf

CPU times: user 2.64 s, sys: 773 ms, total: 3.42 s
Wall time: 3.41 s


,message_timestamp,message_source,message_destination,message_protocol,message_size,coap_type,coap_retransmitted,coap_code,coap_message_id,coap_token,coap_proxy_uri,http.request,http.request.method,http.request.full_uri,http.response,http.response.code,http.response.code.desc,http.response_for.uri,node_type,uid
0,1.647910e+09,client1,proxy,coap,106,con,False,get,1.0,1d8bc74f5569de04,https://10.1.1.1:443/1_1D8BC74F5569DE04,NaN,,,NaN,NaN,,,proxy,1.0_1d8bc74f5569de04
3,1.647910e+09,proxy,client1,coap,158,ack,False,content,1.0,1d8bc74f5569de04,,NaN,,,NaN,NaN,,,proxy,1.0_1d8bc74f5569de04
4,1.647910e+09,client1,proxy,coap,106,con,False,get,2.0,473bbaacedb0fbb2,https://10.1.1.1:443/2_473BBAACEDB0FBB2,NaN,,,NaN,NaN,,,proxy,2.0_473bbaacedb0fbb2
7,1.647910e+09,proxy,client1,coap,158,ack,False,content,2.0,473bbaacedb0fbb2,,NaN,,,NaN,NaN,,,proxy,2.0_473bbaacedb0fbb2
8,1.647910e+09,client1,proxy,coap,106,con,False,get,3.0,091e25b6b4acc2e6,https://10.1.1.1:443/3_091E25B6B4ACC2E6,NaN,,,NaN,NaN,,,proxy,3.0_091e25b6b4acc2e6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964690,1.647911e+09,proxy,client1,coap,158,ack,False,content,1387.0,cdd4686b63c7d8d9,,NaN,,,NaN,NaN,,,proxy,1387.0_cdd4686b63c7d8d9
964691,1.647911e+09,client1,proxy,coap,109,con,False,get,1388.0,f774a0ae8029a854,https://10.1.1.1:443/1388_F774A0AE8029A854,NaN,,,NaN,NaN,,,proxy,1388.0_f774a0ae8029a854
964694,1.647911e+09,proxy,client1,coap,158,ack,False,content,1388.0,f774a0ae8029a854,,NaN,,,NaN,NaN,,,proxy,1388.0_f774a0ae8029a854
964695,1.647911e+09,client1,proxy,coap,109,con,False,get,1389.0,033bbfae19e7c845,https://10.1.1.1:443/1389_033BBFAE19E7C845,NaN,,,NaN,NaN,,,proxy,1389.0_033bbfae19e7c845


In [115]:
print(FILE_PATH)



/home/ubuntu/dos-californium/deter/expdata/real/final/client_and_attacker_httpson/1/proxy_dump.pcap.out


'proxy'

In [77]:
# Validators
assert len(df[~(df["message_timestamp"] > 0)]) == 0
assert len(set(df["message_protocol"].unique()) & {"coap", "http"}) > 0 # At least one of coap, http
assert len(set(df["message_protocol"].unique()) - {"coap", "http"}) == 0 # Nothing but coap, http
assert len(df[~(df["message_size"] > 0)]) == 0

df

,message_timestamp,message_source,message_destination,message_protocol,message_size,coap_type,coap_retransmitted,coap_code,coap_message_id,coap_token,coap_proxy_uri,http.request,http.request.method,http.request.full_uri,http.response,http.response.code,http.response.code.desc,http.response_for.uri
0,1.647910e+09,client1,proxy,coap,106,con,NaN,1.0,1.0,1d8bc74f5569de04,https://10.1.1.1:443/1_1D8BC74F5569DE04,NaN,,,NaN,NaN,,
1,1.647910e+09,proxy,originserver,http,230,NaN,NaN,NaN,NaN,,,1.0,GET,https://10.1.1.1:443/1_1D8BC74F5569DE04,NaN,NaN,,
2,1.647910e+09,originserver,proxy,http,491,NaN,NaN,NaN,NaN,,,NaN,,,1.0,200.0,OK,https://10.1.1.1:443/1_1D8BC74F5569DE04
3,1.647910e+09,proxy,client1,coap,158,ack,NaN,69.0,1.0,1d8bc74f5569de04,,NaN,,,NaN,NaN,,
4,1.647910e+09,client1,proxy,coap,106,con,NaN,1.0,2.0,473bbaacedb0fbb2,https://10.1.1.1:443/2_473BBAACEDB0FBB2,NaN,,,NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964694,1.647911e+09,proxy,client1,coap,158,ack,NaN,69.0,1388.0,f774a0ae8029a854,,NaN,,,NaN,NaN,,
964695,1.647911e+09,client1,proxy,coap,109,con,NaN,1.0,1389.0,033bbfae19e7c845,https://10.1.1.1:443/1389_033BBFAE19E7C845,NaN,,,NaN,NaN,,
964696,1.647911e+09,proxy,originserver,http,233,NaN,NaN,NaN,NaN,,,1.0,GET,https://10.1.1.1:443/1389_033BBFAE19E7C845,NaN,NaN,,
964697,1.647911e+09,originserver,proxy,http,491,NaN,NaN,NaN,NaN,,,NaN,,,1.0,200.0,OK,https://10.1.1.1:443/1389_033BBFAE19E7C845
